In [84]:
import pandas as pd
enriched_address = pd.read_csv('/Users/tamasmakos/dev/survey_methods_room/data_files/enriched_address.csv')

In [85]:
# Printing all columns
pd.set_option('display.max_columns', None)
enriched_address.head()

,id,town_ksh_code,province_name,town_name,town_type1,town_type2,town_type3,district_ksh_code,town_population,street_name,house_number,target,address,geocode,lon,lat,cafe,fuel,restaurant,pub,bank,bus_station,veterinary,pharmacy,fountain,post_office,fast_food,parking,taxi,food_court,dentist,vending_machine,social_facility,parcel_locker,townhall,library,community_centre,bicycle_repair_station,police,bicycle_rental,atm,charging_station,place_of_worship,toilets,clock,ice_cream,chair,doctors,kindergarten,childcare,rental,bum,parking_entrance,nightclub,marketplace,bar,car_wash,school,biergarten,language_school,bbq,smoking_area,arts_centre,cinema,bureau_de_change,courthouse,clinic,shelter,printer,parking_space,studio,motorcycle_parking,hunting_stand,escooter_rental,coworking_space,hospital,vehicle_ramp,driving_school,theatre,waste_disposal,air_filling,mist_spraying_cooler,art_school,nursing_home,car_rental,compressed_air,dojo,public_hall,table,internet_cafe,playhouse,dog_toilet,ferry_terminal,workshop,music_school,university,public_bookcase,college,dancing_school,casino,ticket_validator,trolley_bay,money_transfer,dressing_room,water_point,gambling,lounger,weighbridge,prison,funeral_hall,fire_station,migrant_detention_centre,food,vacuum_cleaner,boat_rental,animal_boarding,vehicle_inspection,shower,social_centre,monastery,ski_rental,yes,office,grit_bin,locker,music_venue,ritual_bath,hookah_lounge,training,dormitory,sanitary_dump_station,device_charging_station,left_luggage,car_sharing,luggage_locker,karaoke_box,stripclub,n,m,k_avg,edge_length_total,edge_length_avg,streets_per_node_avg,streets_per_node_counts,streets_per_node_proportions,intersection_count,street_length_total,street_segment_count,street_length_avg,circuity_avg,self_loop_proportion,clean_intersection_count,degree_centrality,closeness_centrality,betweenness_centrality,eigenvector_centrality,clustering_coefficient,pagerank
0,1,27872,13,Abony,3,2,3,124,14916,attila utca,27,0,"attila utca 27, Abony","{'x': 20.014630015946, 'y': 47.193619980101}",20.01463,47.19362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0,2.666667,402.460,100.615000,3.000000,"{0: 0, 1: 0, 2: 0, 3: 3}","{0: 0.0, 1: 0.0, 2: 0.0, 3: 1.0}",3.0,201.230,2.0,100.615000,1.000027,0.0,1.0,1.000000,0.666667,0.000000,0.500000,0.0,0.256757
1,2,27872,13,Abony,3,2,3,124,14916,bicskei út,4,1,"bicskei út 4, Abony","{'x': 19.994960036126, 'y': 47.189520014073}",19.99496,47.18952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,14.0,4.000000,1251.724,89.408857,3.000000,"{0: 0, 1: 0, 2: 0, 3: 7}","{0: 0.0, 1: 0.0, 2: 0.0, 3: 1.0}",7.0,625.862,7.0,89.408857,1.009387,0.0,1.0,0.666667,0.461538,0.066667,0.358781,0.0,0.139770
2,3,27872,13,Abony,3,2,3,124,14916,csiky gergely utca,20,1,"csiky gergely utca 20, Abony","{'x': 20.010869978001, 'y': 47.182610015004}",20.01087,47.18261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [86]:
# Removing columns
summary = enriched_address.describe().transpose()

# Rows where values are 1
columns_to_delete = summary[['25%', '50%', '75%']][summary[['25%', '50%', '75%']]==1]

# Removing NaN 
columns_to_delete = columns_to_delete.dropna().transpose().columns

# Removing columns
enriched_address = enriched_address.drop(columns_to_delete, axis=1)

In [87]:
import os
import pandas as pd
import logging

# Set up logging
logging.basicConfig(filename='log.txt', level=logging.ERROR, format='%(asctime)s %(levelname)s %(message)s')

# Create an empty dataframe to store the data
df = pd.DataFrame()

# Loop through all xlsx files in the directory
for file in os.listdir('/Users/tamasmakos/dev/survey_methods_room/data_files/real_estate_pices'):
    if file.endswith('.xlsx'):
        try:
            # Read the xlsx file
            xlsx = pd.read_excel('/Users/tamasmakos/dev/survey_methods_room/data_files/real_estate_pices/' + file, sheet_name=None, skiprows=1, header=[0,1,2], engine='openpyxl')
            
            # List to store dataframes from each sheet
            df_list = []
            
            for name in xlsx.keys():
                if name != 'Tartalom':
                    # Clean up the headers to have a uniform structure
                    sheet_df = xlsx[name]
                    sheet_df.columns = ['_'.join(col).strip() for col in sheet_df.columns.values]
                    
                    # Add the cleaned dataframe to the list
                    df_list.append(sheet_df)
            
            # Concatenate the cleaned sheets into one dataframe with axis=0
            df_temp = pd.concat(df_list, axis=0)
            
            # Add a column with the name of the file
            df_temp['file_name'] = file
            
            # Append the dataframe to the main dataframe with axis=0
            df = pd.concat([df, df_temp], axis=0)
        except Exception as e:
            # Log the error
            logging.error(f"Error processing file {file}: {e}")
            continue

# Display the first few rows of the dataframe
df.head()


,Az ingatlan helye_Unnamed: 0_level_1_Unnamed: 0_level_2,Az ingatlan helye_Unnamed: 1_level_1_Unnamed: 1_level_2,"Az ingatlan típusa_családi ház_átlagár, ezer Ft/m2",Az ingatlan típusa_családi ház_darab,"Az ingatlan típusa_többlakásos társasház_átlagár, ezer Ft/m2",Az ingatlan típusa_többlakásos társasház_darab,"Az ingatlan típusa_panel_átlagár, ezer Ft/m2",Az ingatlan típusa_panel_darab,"Lakások összesen_Unnamed: 8_level_1_átlagár, ezer Ft/m2",Lakások összesen_Unnamed: 9_level_1_darab,"Lakások összesen_Unnamed: 10_level_1_relatív szórás, %",file_name,"Az ingatlan típusa_lakótelepi panel_átlagár, ezer Ft/m2",Az ingatlan típusa_lakótelepi panel_darab
0,Budapest 01. ker.,Aladár utca,-,-,253,9,-,-,252.778761,9.0,29,inga2013.xlsx,NaN,NaN
1,Budapest 01. ker.,Attila út,-,-,240,54,-,-,239.787157,54.0,25,inga2013.xlsx,NaN,NaN
2,Budapest 01. ker.,Batthyány utca,-,-,283,25,-,-,283.238108,25.0,25,inga2013.xlsx,NaN,NaN
3,Budapest 01. ker.,Csalogány utca,-,-,312,19,-,-,311.797341,19.0,19,inga2013.xlsx,NaN,NaN
4,Budapest 01. ker.,Derék utca,-,-,293,3,-,-,293.294662,3.0,18,inga2013.xlsx,NaN,NaN


In [88]:
# Selecting specific columns 0,1 and 8
df = df.iloc[:, [0,1,8,11]]
df.head()

,Az ingatlan helye_Unnamed: 0_level_1_Unnamed: 0_level_2,Az ingatlan helye_Unnamed: 1_level_1_Unnamed: 1_level_2,"Lakások összesen_Unnamed: 8_level_1_átlagár, ezer Ft/m2",file_name
0,Budapest 01. ker.,Aladár utca,252.778761,inga2013.xlsx
1,Budapest 01. ker.,Attila út,239.787157,inga2013.xlsx
2,Budapest 01. ker.,Batthyány utca,283.238108,inga2013.xlsx
3,Budapest 01. ker.,Csalogány utca,311.797341,inga2013.xlsx
4,Budapest 01. ker.,Derék utca,293.294662,inga2013.xlsx


In [89]:
# Renaming columns
df.columns = ['town', 'street', 'average_price/sqm', 'year']

In [90]:
# Selecting the year from year with a regex
df['year'] = df['year'].str.extract(r'(\d{4})')

In [91]:
df.head()

,town,street,average_price/sqm,year
0,Budapest 01. ker.,Aladár utca,252.778761,2013
1,Budapest 01. ker.,Attila út,239.787157,2013
2,Budapest 01. ker.,Batthyány utca,283.238108,2013
3,Budapest 01. ker.,Csalogány utca,311.797341,2013
4,Budapest 01. ker.,Derék utca,293.294662,2013


In [104]:
# Dropping all rows where the town name is not in enriched_address
df = df[df['town'].isin(enriched_address['town_name'])]

df.shape

(63224, 4)

In [105]:
# Unique combinations of town and street
geo = df[['town', 'street']].drop_duplicates()

geo.shape

(13465, 2)

In [ ]:

import requests
def geocode(address):
    url = 'https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/findAddressCandidates'
    params = {'f': 'json', 'singleLine': address, 'outFields': 'Match_addr,Addr_type'}
    r = requests.get(url, params=params)
    if r.json()['candidates']:
        return r.json()['candidates'][0]['location']
    else:
        return None


# Geocoding geo town and steet combinations
geo['location'] = geo.apply(lambda x: geocode(x['town'] + ', ' + x['street']), axis=1)
geo['lat'] = geo['location'].apply(lambda x: x['y'] if x else None)
geo['lon'] = geo['location'].apply(lambda x: x['x'] if x else None)